In [1]:
import tensorflow as tf
from time import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#import mlutils
%matplotlib inline

### descargamos imágenes de dos queries en google

In [2]:
downloads = "/tmp/downloads"
traintest_imgs = "/tmp/imgs"

train_pct = .7

In [3]:
# !rm -rf $downloads
# !mkdir $downloads

In [4]:
# cfg="""
# {
#     "Records": [
#         {
#             "keywords": "fire",
#             "format": "jpg",
#             "limit": 100,
#             "type": "photo",
#             "size": "medium",
#             "output_directory": "%s"
#         },
#         {
#             "keywords": "random",
#             "format": "jpg",
#             "limit": 100,
#             "type": "photo",
#             "size": "medium", 
#             "output_directory": "%s"
#         }
#     ]
# }
# """%(downloads, downloads)

In [5]:
# with open(downloads+"/cfg.txt", "w") as f:
#     f.write(cfg)

In [6]:
# !googleimagesdownload --config_file $downloads/cfg.txt

### revisamos las imágenes y eliminamos las defectuosas

In [7]:
# from skimage import io
# files = !find $downloads -type f
# for fname in files:
#     if fname!=downloads+"/cfg.txt":
#         print (".",)
#         try:
#             io.imread(fname)
#         except:
#             print ("\nremoving",fname,"as cannot be read\n")
#             !rm $fname

### organizamos las imágenes en train y en test

In [8]:
classdirs = !find $downloads -type d
classdirs = [i.split("/")[-1] for i in classdirs if i!=downloads]
print (classdirs)

['fire', 'random']


In [9]:
!rm -rf $traintest_imgs

In [10]:
for classdir in classdirs:
    print (classdir)
    print ("    splitting files ... "),
    files = !find $downloads/$classdir -type f
    files = np.r_[files]
    n_train = int(len(files)*train_pct)
    files = np.random.permutation(files)
    files_train = files[:n_train]
    files_test  = files[n_train:]
    !mkdir -p $traintest_imgs/train/$classdir
    !mkdir -p $traintest_imgs/test/$classdir
    print ("copying files")      
    for f in files_train:
        #print (f)
        !cp $f $traintest_imgs/train/$classdir
    for f in files_test:
        #print (f)
        !cp $f $traintest_imgs/test/$classdir

fire
    splitting files ... 
copying files
/bin/sh: 1: Syntax error: "(" unexpected
random
    splitting files ... 
copying files


### sanity check 1

In [11]:
for classdir in classdirs:
    print (classdir)
    files = !find  $traintest_imgs/train/$classdir -type f
    print ("      ",len(files), "train images")
    files = !find  $traintest_imgs/test/$classdir -type f
    print ("      ",len(files), "test images")

fire
       59 train images
       26 test images
random
       56 train images
       24 test images


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [13]:
datagen = ImageDataGenerator(rescale=1./255, fill_mode='nearest')
batch_size = 10
img_size = 64
train_generator_fn = lambda: datagen.flow_from_directory(
        traintest_imgs+"/train",  # this is the target directory
        target_size=(img_size, img_size),  # all images will be resized to 150x150
        batch_size=batch_size,
        interpolation = "bilinear",
        class_mode='binary') 

test_generator_fn = lambda: datagen.flow_from_directory(
        traintest_imgs+"/test",  # this is the target directory
        target_size=(img_size, img_size),  # all images will be resized to 150x150
        batch_size=batch_size,
        interpolation = "bilinear",
        class_mode='binary') 

### train keras!!!

In [14]:
def get_conv_model_A(num_classes, img_size=64, compile=True):
    from tensorflow.python.framework import ops
    ops.reset_default_graph()
    tf.keras.backend.clear_session()
    print ("using",num_classes,"classes")
    inputs = tf.keras.Input(shape=(img_size,img_size,3), name="input_1")
    layers = tf.keras.layers.Conv2D(15,(10,10), activation="relu")(inputs)
    layers = tf.keras.layers.Flatten()(layers)
    layers = tf.keras.layers.Dense(64, activation=tf.nn.relu)(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    predictions = tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax, name="output_1")(layers)
    model = tf.keras.Model(inputs = inputs, outputs=predictions)
    if compile:
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
    return model

In [15]:
num_classes = len(classdirs)
model = get_conv_model_A(num_classes, img_size=img_size)

model.fit_generator(
        train_generator_fn(),
        steps_per_epoch=1000 // batch_size,
        epochs=5,
        validation_data=test_generator_fn(),
        validation_steps=400 // batch_size)


# model.fit(
#         train_generator_fn(),
#         steps_per_epoch=1000 // batch_size,
#         epochs=3,
#         validation_data=test_generator_fn(),
#         validation_steps=400 // batch_size)

using 2 classes
Found 115 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
Epoch 1/5
100/100 [==============================] - 16s 159ms/step - loss: 0.4838 - accuracy: 0.8021 - val_loss: 0.8729 - val_accuracy: 0.7400
Epoch 2/5
100/100 [==============================] - 15s 151ms/step - loss: 0.0985 - accuracy: 0.9677 - val_loss: 1.3635 - val_accuracy: 0.7600
Epoch 3/5
100/100 [==============================] - 15s 152ms/step - loss: 0.0403 - accuracy: 0.9874 - val_loss: 1.3785 - val_accuracy: 0.7600
Epoch 4/5
100/100 [==============================] - 15s 151ms/step - loss: 0.0139 - accuracy: 0.9948 - val_loss: 1.4803 - val_accuracy: 0.7800
Epoch 5/5
100/100 [==============================] - 15s 150ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 1.7957 - val_accuracy: 0.7800


In [16]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
